Hello.

I am a data science professional, and spend my weekends on Kaggle.

Sharing an amazing technique I use often in business scenario, to validate if the train and test sets are similar or not. Since most beginners come here with the intent of landing a job, hope this helps you.

**This is an artificially generated data, so the check is not needed, but tis always good to check before you begin modelling**

** Please upvote this Kernel if you find it useful **

<h1> Adversarial Validation </h1>
Drawing a Parallel from Adversarial networks, We make parts of dataset compete against each other to analyse for differences and similarities.
Simply put, these are the steps for Adversarial Validation:


1. Combine the training and testing dataset.Create a flag that indicates the source dataset.
2. Train a base classifier on the created flag
3. Analyze AUC, accuracy etc to see how similar training and testing datasets are. 

**Generally, The closer AUC is to .5, the better it is.**

<h1> Implications </h1>

Adversarial Validation can help us analyze whether a data science problem can be formulated or not. It also checks for data stability.
In business context, AV is a great analysis to apply at the EDA phase. Any situation where the characteristics of Train and Test data itself are very different, ML might not be the best choice to solve a problem.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Reading Data
train_data = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')


test_data = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')

<h1> Creating Flags </h1>

In [ ]:
#Creating flags
train_data['istrain'] = 1

test_data['istrain'] = 0

<h1> Combining Data </h1>

In [ ]:
#Combining Data
combined_data = pd.concat([train_data, test_data], axis = 0)

In [ ]:
combined_data['istrain'].value_counts()

In [ ]:
df_numeric = combined_data.select_dtypes(exclude=['object'])

df_obj = combined_data.select_dtypes(include=['object']).copy()
    
for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]
    
combined_data = pd.concat([df_numeric, df_obj], axis=1)

y = combined_data['istrain']

#dropping identifiers and target. 

combined_data.drop(['id','istrain','target'], axis = 1, inplace = True)

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
combined_data.shape

<h1> Model training </h1>

In [ ]:
#I'll use an XGB classifier here with default parameters, you may use anything

skf = StratifiedShuffleSplit()
xgb_params = {'objective': 'binary:logistic','use_label_encoder':False,'n_estimators':100}   
clf = xgb.XGBClassifier(**xgb_params, seed = 10)    

In [ ]:
scores = []

avg_loss = []

for train_index, test_index in skf.split(combined_data, y):
       
        x0, x1 = combined_data.iloc[train_index], combined_data.iloc[test_index]
        
        y0, y1 = y.iloc[train_index], y.iloc[test_index]        
        
        print(x0.shape)
        
        clf.fit(x0, y0, eval_set=[(x1, y1)],
               eval_metric=['logloss','auc'], verbose=True,early_stopping_rounds=50)
                
        prval = clf.predict_proba(x1)[:,1]
        
        roc = roc_auc_score(y1,prval)

        scores.append(roc)
        
        avg_loss.append(clf.best_score)

        print ('XGB Val OOF AUC=',roc)
    
print("Log Loss Stats {0:.8f},{1:.8f}".format(np.array(avg_loss).mean(), np.array(avg_loss).std()))

<h1 Moment of truth :AUC </h1

In [ ]:
print('%.8f (%.8f)' % (np.array(scores).mean(), np.array(scores).std()))

<h1> Interpretation </h1>

AUC score is close to .5 here, implying the train and test datasets are not distinguishable.

In case of a high AUC, This means normal validation techniques might not work well here.

It can also indicate the case of a target leakage for train dataset.

It can be used to identify anomalies and refine training set if needed.

<h1> Feature importance </h1>

Looking at what features are helping the model distinguish the best,so we may drop it.

In [ ]:
# print(clf.feature_importances_)

In [ ]:
from xgboost import plot_importance
from matplotlib import pyplot
from pylab import rcParams
rcParams['figure.figsize'] = 20,10
plot_importance(clf,max_num_features=20) #top 20 features only
pyplot.show()


<h1> References </h1>

1. https://www.kaggle.com/gcspkmdr/tabular-playground-mar-adversarial-validation

2. https://www.kaggle.com/c/widsdatathon2021/discussion/222991

3. https://machinelearningmastery.com/feature-importance-and-feature-selection-with-xgboost-in-python/

4. https://www.kaggle.com/kritidoneria/wids-2021-adversarial-validation-on-train-test
